# 과제

데이터셋을 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 torch.utils.data.Dataset과 torch.utils.data.DataLoader를 제공 한다.  
 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있다. 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것이다
 
# 커스텀 데이터셋(Custom Dataset)  
torch.utils.data.Dataset을 상속받아 직접 커스텀 데이터셋(Custom Dataset)을 만들 수 있다.  
Dataset을 상속받아 다음 메소드들을 오버라이드 하여 커스텀 데이터셋을 생성해 보자.  
커스텀 데이터셋을 만들 때, 일단 가장 기본적인 뼈대는 아래와 같다. 
```
class CustomDataset(torch.utils.data.Dataset): 
  def __init__(self):

  def __len__(self):

  def __getitem__(self, idx): 
```

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

### Diabetes dataset
Diabetes dataset은 총 442명의 당뇨병 환자에 대한 자료이다.  
age, sex, body mass index, average blood pressure, 6개의 혈청값으로 이루어져 있다.  
442명의 당뇨병 환자를 대상으로한 검사 결과를 나타내는 데이터이다.

- 타겟 데이터 : 1년 뒤 측정한 당뇨병의 진행률

- 특징 데이터 (이 데이터셋의 특징 데이터는 모두 정규화된 값이다.)

  - Age
  - Sex
  - Body mass index
  - Average blood pressure
  - S1
  - S2
  - S3
  - S4
  - S5
  - S6


In [2]:
df = pd.read_csv('diabetes.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


### 문제 1
다음을 참조하여 `diabetes.csv` 파일을 읽고 custom dataset으로 생성해 보시오.

* len(dataset)을 했을 때 데이터셋의 크기를 리턴할 len
* dataset[i]을 했을 때 i번째 샘플을 가져오도록 하는 인덱싱을 위한 get_item

# TODO : 다음의 코드를 완성하시오.
class CustomDataset(torch.utils.data.Dataset): 
    def __init__(self):
    # 여기에 코드를 작성하시오.
    def __len__(self):
    # 여기에 코드를 작성하시오.
    def __getitem__(self, idx): 
    # 여기에 코드를 작성하시오.

In [3]:
class CustomDataset(Dataset):
    def __init__(self):
        data = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.x_data = torch.from_numpy(data[:, :-1]) # age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
        self.y_data = torch.from_numpy(data[:, -1]).view(-1,1) # target
    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y
    def __len__(self):
        return len(self.x_data)

### 문제 2
dataset을 생성하고 `__getitem__()` 속성을 사용하여 데이터를 조회해 보시오.

In [4]:
dataset = CustomDataset()

dataset.__getitem__([0])

(tensor([[ 0.0381,  0.0507,  0.0617,  0.0219, -0.0442, -0.0348, -0.0434, -0.0026,
           0.0199, -0.0176]]),
 tensor([[151.]]))

### 문제 3
읽어들인 데이셋을 신경망 모형의 입력에 사용할 수 있도록 `DataLoader` 를 사용하시오.  
적절한 batch_size를 설정하시오.

In [5]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

x, y = iter(dataloader).next()

print(x.shape, y.shape)

torch.Size([10, 10]) torch.Size([10, 1])


### 문제 4
diabetes를 예측하는 신경망 모형을 생성하시오.

In [10]:
import torch
from torch import nn

model = nn.Sequential(nn.Linear(10, 1))
print(model)

Sequential(
  (0): Linear(in_features=10, out_features=1, bias=True)
)


### 문제 5

loss 함수와 optimizer를 설정하시오. learning_rate를 적절히 선택하시오.

In [7]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001) 

### 문제 6
모형을 훈련하시오. epoch의 횟수를 적절히 선택하시오.

In [8]:
epochs = 1000

for epoch in range(epochs):
    running_loss = 0
    for data, target in dataloader:
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        if epoch % 100 == 0 :
            print(f"Training loss: {running_loss/len(dataloader)}")

Training loss: 28609.126323784723
Training loss: 9504.185909016927
Training loss: 6431.6843044704865
Training loss: 5825.768329196506
Training loss: 5974.438216145833
Training loss: 5753.489246961805
Training loss: 5752.729806857639
Training loss: 5658.602563476563
Training loss: 5636.333854166666
Training loss: 5628.204378255208


### 문제 7
```
new_var =  torch.FloatTensor([[ 0.0381,  0.0507,  0.0617,  0.0219, -0.0442, -0.0348, 
                               -0.0434, -0.0026, 0.0199, -0.0176]])
```
새로운 데이터로 diabets를 예측해 보시오.

In [9]:
new_var =  torch.FloatTensor([[ 0.0381,  0.0507,  0.0617,  0.0219, -0.0442, -0.0348, 
                               -0.0434, -0.0026, 0.0199, -0.0176]])
with torch.no_grad():
    y_hat = model(new_var)
    
print(y_hat)

tensor([[153.9861]])
